## Git Theory

### The revision Graph

Revisions form a **GRAPH**

In [1]:
import os
top_dir = os.getcwd()
git_dir = os.path.join(top_dir, 'learning_git')
working_dir=os.path.join(git_dir, 'git_example')
os.chdir(working_dir)

In [3]:
%%bash
git log --graph --oneline

*   c8ab86a Merge branch 'master' of github.com:UCL/github-example
|\  
| * 6049543 Add another Beacon
* | 42f4945 Add Glyder
|/  
*   8e3a6b5 Merge branch 'master' of github.com:UCL/github-example
|\  
| * 76f8c93 Add a beacon
* | e19b456 Translating from the Welsh
|/  
*   39cdf96 Merge branch 'master' of github.com:UCL/github-example
|\  
| * fea6f2c Add Scotland
* | 740c880 Add wales
|/  
* ee4fef3 Add Helvellyn
* 1dc2b88 Include lakes in the scope
* de6bec9 Add lakeland
* 4712e37 Revert "Add a lie about a mountain"
* a98be01 Change title
* faa14cf Add a lie about a mountain
* 75d400d First commit of discourse on UK topography


### Git concepts

* Each revision has a parent that it is based on
* These revisions form a graph
* Each revision has a unique hash code
  * In Sue's copy, revision 43 is ab3578d6
  * Jim might think that is revision 38, but it's still ab3579d6
* Branches, tags, and HEAD are *labels* pointing at revisions
* Some operations (like fast forward merges) just move labels.

### The levels of Git

There are four **Separate** levels a change can reach in git:

* The Working Copy
* The **index** (aka **staging area**)
* The local repository
* The remote repository

Understanding all the things `git reset` can do requires a good
grasp of git theory.

* `git reset <commit> <filename>` : Reset index and working version of that file to the version in a given commit
* `git reset --soft <commit>`: Move local repository branch label to that commit, leave working dir and index unchanged
* `git reset <commit>`: Move local repository and index to commit ("--mixed")
* `git reset --hard <commit>`: Move local repostiory, index, and working directory copy to that state

In [6]:
## Git Stash

If you find you want to pull, but you're not ready to commit,
you can use

In [10]:
%%writefile Wales.md
Mountains In Wales
==================

* Pen y Fan
* Tryfan
* Snowdon
* Glyder Fawr
* Fan y Big
* Cadair Idris

Overwriting Wales.md


In [14]:
%%bash
git stash
git pull

No local changes to save
Already up-to-date.


In [15]:
%%bash
git stash apply

On branch master
Your branch is up-to-date with 'origin/master'.
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Wales.md

no changes added to commit (use "git add" and/or "git commit -a")


The "Stash" is a way of temporarily saving your working area, and can help out in a pinch.

## Branches

Branches are increadibly important to why `git` is cool and powerful.

They are an easy and cheap way of making a second version of your software, which you work on in parallel,
and pull in your changes when you are ready.

In [16]:
%%bash
git branch # Tell me what branches exist

* master


In [17]:
%%bash
git checkout -b experiment # Make a new branch

M	Wales.md


Switched to a new branch 'experiment'


In [18]:
%%bash
git branch

* experiment
  master


In [19]:
%%bash
git commit -am "Add Cadair Idris"

[experiment d80ba89] Add Cadair Idris
 1 file changed, 2 insertions(+), 1 deletion(-)


In [20]:
%%bash
git checkout master # Switch to an existing branch

Your branch is up-to-date with 'origin/master'.


Switched to branch 'master'


In [21]:
%%bash
cat Wales.md

Mountains In Wales

* Pen y Fan
* Tryfan
* Snowdon
* Glyder Fawr
* Fan y Big

In [23]:
%%bash
git checkout experiment

Already on 'experiment'


In [24]:
cat Wales.md

Mountains In Wales

* Pen y Fan
* Tryfan
* Snowdon
* Glyder Fawr
* Fan y Big
* Cadair Idris

### Publishing branches

To let the server know there's a new branch use:

In [27]:
%%bash
git push -u origin experiment

Branch experiment set up to track remote branch experiment from origin.


To git@github.com:UCL/github-example.git
 * [new branch]      experiment -> experiment


We use `--set-upstream origin` (Abbreviation `-u`) to tell git that this branch should be pushed to and pulled from origin per default.

If you are following along, you should be able to see your branch in the list of branches in GitHub.

Once you've used `git push -u` once, you can push new changes to the branch with just a git push.

If others checkout your repository, they will be able to do `git checkout experiment` to see your branch content,
and collaborate with you **in the branch**.

In [28]:
%%bash
git branch -r

  origin/experiment
  origin/master


Local branches can be, but do not have to be, connected to remote branches
They are said to "track" remote branches. `push -u` sets up the tracking relationship.

In [29]:
%%bash
git branch -vv

* experiment d80ba89 [origin/experiment] Add Cadair Idris
  master     c8ab86a [origin/master] Merge branch 'master' of github.com:UCL/github-example


### Find out what is on a branch

In addition to using `git diff` to compare to the state of a branch,
you can use `git log` to look at lists of commits which are in a branch
and haven't been merged yet.

In [32]:
%%bash
git log master..experiment

commit d80ba892687b33b2b01a91cca7b72c18ff0c4107
Author: James Hetherington <jamespjh@gmail.com>
Date:   Wed Sep 2 18:15:56 2015 +0100

    Add Cadair Idris


Git uses various symbols to refer to sets of commits.
The double dot `A..B` means "ancestor of B and not ancestor of A"

So in a purely linear sequence, it does what you'd expect.

In [89]:
%%bash
git log --graph --oneline HEAD~9..HEAD~5

* 740c880 Add wales
* ee4fef3 Add Helvellyn
* 1dc2b88 Include lakes in the scope
* de6bec9 Add lakeland


But in cases where a history has branches,
the definition in terms of ancestors is important.

In [86]:
%%bash
git log --graph --oneline HEAD~5..HEAD

*   c8ab86a Merge branch 'master' of github.com:UCL/github-example
|\  
| * 6049543 Add another Beacon
* | 42f4945 Add Glyder
|/  
*   8e3a6b5 Merge branch 'master' of github.com:UCL/github-example
|\  
| * 76f8c93 Add a beacon
* | e19b456 Translating from the Welsh
|/  
* 39cdf96 Merge branch 'master' of github.com:UCL/github-example
* fea6f2c Add Scotland


If there are changes on both sides, like this:

In [ ]:
%%bash
git checkout master

In [63]:
%%writefile Scotland.md
Mountains In Scotland
==================

* Ben Eighe
* Cairngorm
* Aonach Eagach


Overwriting Scotland.md


In [64]:
%%bash
git diff Scotland.md

diff --git a/Scotland.md b/Scotland.md
index 36f83a1..44eb7ea 100644
--- a/Scotland.md
+++ b/Scotland.md
@@ -2,4 +2,5 @@ Mountains In Scotland
 
 * Ben Eighe
-* Cairngorm
\ No newline at end of file
+* Cairngorm
+* Aonach Eagach
\ No newline at end of file


In [93]:
%%bash
git commit -am "Commit Aonach onto master branch"

[master d92c3d6] Commit Aonach onto master branch
 1 file changed, 2 insertions(+), 1 deletion(-)


Then this notation is useful to show the content of what's on what branch:

In [94]:
%%bash
git log --left-right --oneline master...experiment

< d92c3d6 Commit Aonach onto master branch
> d80ba89 Add Cadair Idris


Three dots means "everything which is not a common ancestor" of the two commits, i.e. the differences between them.

### Merging branches

We can merge branches, and just as we would pull in remote changes, there may or may not be conflicts.

In [100]:
%%bash
git branch
git merge experiment

  experiment
* master
Already up-to-date.


In [104]:
%%bash
git log --graph --oneline HEAD~3..HEAD

*   762040e Merge branch 'experiment'
|\  
| * d80ba89 Add Cadair Idris
* | d92c3d6 Commit Aonach onto master branch
|/  
* c8ab86a Merge branch 'master' of github.com:UCL/github-example
* 6049543 Add another Beacon


### Cleaning up after a branch

In [105]:
%%bash
git branch

  experiment
* master


In [106]:
%%bash
git branch -d experiment

Deleted branch experiment (was d80ba89).


In [107]:
%%bash
git branch

* master


In [110]:
%%bash
git branch --remote

  origin/experiment
  origin/master


In [115]:
%%bash
git push --delete origin experiment # Remove remote branch - also can use github interface

To git@github.com:UCL/github-example.git
 - [deleted]         experiment


In [117]:
%%bash
git branch --remote

  origin/master


### A good branch strategy

* A `production` branch: code used for active work
* A `develop` branch: for general new code
* `feature` branches: for specific new ideas
* `release` branches: when you share code with others
  * Useful for isolated bug fixes

### Grab changes from a branch

Make some changes on one branch, switch back to another, and use:

```bash
git checkout <branch> <path>
```

To quickly grab a file from one branch into another.

Using `git checkout` with a path takes the content of files.
To grab the content of a specific *commit* from another branch,
and apply it as a patch to your branch, use:

``` bash
git cherry-pick <commit>
```

## Tagging

Tags are easy to read labels for revisions, and can be used anywhere we would name a commit.

Produce real results *only* with tagged revisions

In [120]:
%%bash
git tag -a v1.0 -m "Release 1.0"
git push --tags

To git@github.com:UCL/github-example.git
 * [new tag]         v1.0 -> v1.0


In [126]:
%%writefile Pennines.md

Mountains In the Pennines
========================

* Cross Fell

Writing Pennines.md


In [127]:
%%bash
git add Pennines.md
git commit -am "Add Pennines"

[master d155c63] Add Pennines
 1 file changed, 5 insertions(+)
 create mode 100644 Pennines.md


In [129]:
%%bash
git log v1.0.. --graph --oneline

* d155c63 Add Pennines


If .. is used without a following commit name, HEAD is assumed.

## Working with generated files

We often end up with files that are generated by our program. It is bad practice to keep these in Git; just keep the sources.

Examples include `.o` and `.x` files for compiled languages, `.pyc` files in Python.

In our example, we might want to make our .md files into a PDF with pandoc:

In [158]:
%%writefile Makefile

MDS=$(wildcard *.md)
PDFS=$(MDS:.md=.pdf)

default: $(PDFS)

%.pdf: %.md
	pandoc $< -o $@

Overwriting Makefile


In [159]:
%%bash
make

make: Nothing to be done for `default'.


We now have a bunch of output .pdf files corresponding to each Markdown file.

But we don't want those to show up in git:

In [160]:
%%bash
git status

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Makefile
	Pennines.pdf
	Scotland.pdf
	Wales.pdf
	index.pdf
	lakeland.pdf

nothing added to commit but untracked files present (use "git add" to track)


### Gitignore

Use .gitignore files to tell Git not to pay attention to files with certain paths:

In [163]:
%%writefile .gitignore
*.pdf

Overwriting .gitignore


In [164]:
%%bash
git status

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.gitignore
	Makefile

nothing added to commit but untracked files present (use "git add" to track)


In [167]:
%%bash
git add Makefile
git add .gitignore
git commit -am "Add a makefile and ignore generated files"
git push

[master 296f14c] Add a makefile and ignore generated files
 2 files changed, 9 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 Makefile


### Git clean

In [173]:
%%bash
git clean -fX

In [174]:
%%bash
ls

Makefile
Pennines.md
Scotland.md
Wales.md
index.md
lakeland.md


* With -f: don't prompt
* with -d: remove directories
* with -x: Also remote .gitignored files
* with -X: Only remove .gitignore files

## Working with multiple remotes

### Distributed versus centralised

Older version control systems (cvs, svn) were "centralised"; the history was kept only on a server,
and all commits required an internet.

Centralised                    |  Distributed
-------------------------------|--------------------------
Server has history             |Every user has full history
Your computer has one snapshot |  Many local branches
To access history, need internet| History always available
You commit to remote server     | Users synchronise histories
cvs, subversion(svn)            | git, mercurial (hg), bazaar (bzr)

With modern distributed systems, we can add a second remote. This might be a personal *fork* on github:

In [179]:
%%bash
git remote add jamespjh git@github.com:jamespjh/github-example.git
git remote -v

jamespjh	git@github.com:jamespjh/github-example.git (fetch)
jamespjh	git@github.com:jamespjh/github-example.git (push)
origin	git@github.com:UCL/github-example.git (fetch)
origin	git@github.com:UCL/github-example.git (push)


fatal: remote jamespjh already exists.


We can push to a named remote:

In [180]:
%%writefile Pennines.md

Mountains In the Pennines
========================

* Cross Fell
* Whernside

Overwriting Pennines.md


In [181]:
%%bash
git commit -am "Add Whernside"

[master 5e25fe8] Add Whernside
 1 file changed, 2 insertions(+), 1 deletion(-)


In [182]:
%%bash
git push jamespjh

To git@github.com:jamespjh/github-example.git
   762040e..5e25fe8  master -> master


### Referencing remotes

You can always refer to commits on a remote like this:

In [187]:
%%bash
git fetch
git log --oneline --left-right jamespjh/master...origin/master

< 5e25fe8 Add Whernside
< 296f14c Add a makefile and ignore generated files
< d155c63 Add Pennines


To see the differences between remotes, for example.

To see what files you have changed that aren't updated on a particular remote, for example:

In [190]:
%%bash
git diff --name-only origin/master

.gitignore
Makefile
Pennines.md


When you reference remotes like this, you're working with a cached copy of the last time you interacted with the remote. You can do `git fetch` to update local data with the remotes without actually pulling. You can also get useful information about whether tracking branches are ahead or behind the remote breanches they track:

In [193]:
%%bash
git branch -vv

* master 5e25fe8 [origin/master: ahead 3] Add Whernside


## Hosting Servers

### Hosting a local server

* Any repository can be a remote for pulls
* Can pull/push over shared folders or ssh
* Pushing to someone's working copy is dangerous
* Use `git init --bare` to make a copy for pushing
* You don't need to create a "server" as such, any 'bare' git repo will do.

In [197]:
bare_dir=os.path.join(git_dir, 'bare_repo')
os.chdir(git_dir)

In [198]:
%%bash
mkdir -p bare_repo
cd bare_repo
git init --bare

Initialized empty Git repository in /Users/jamespjh/devel/rsdt/rsd-engineeringcourse/ch02git/learning_git/bare_repo/


In [199]:
os.chdir(working_dir)

In [201]:
%%bash
git remote add local_bare ../bare_repo
git push local_bare

fatal: remote local_bare already exists.
To ../bare_repo
 * [new branch]      master -> master


In [205]:
%%bash
git remote -v

jamespjh	git@github.com:jamespjh/github-example.git (fetch)
jamespjh	git@github.com:jamespjh/github-example.git (push)
local_bare	../bare_repo (fetch)
local_bare	../bare_repo (push)
origin	git@github.com:UCL/github-example.git (fetch)
origin	git@github.com:UCL/github-example.git (push)


You can now work with this local repository, just as with any other git server.
If you have a colleague on a shared file system, you can use this approach to collaborate through that file system.

### Home-made SSH servers

Classroom exercise: Try creating a server for yourself using a machine you can SSH to:

``` bash
ssh <mymachine>
mkdir mygitserver
cd mygitserver
git init --bare
exit
git remote add <somename> ssh://user@host/mygitserver
git push -u <somename> master
```

## Hunks

### Git Hunks

A "Hunk" is one git change. This changeset has three hunks:

```python
+import matplotlib
+import numpy as np

 from matplotlib import pylab
 from matplotlib.backends.backend_pdf import PdfPages

+def increment_or_add(key,hash,weight=1):
+       if key not in hash:
+               hash[key]=0
+       hash[key]+=weight
+
 data_path=os.path.join(os.path.dirname(
                        os.path.abspath(__file__)),
-regenerate=False
+regenerate=True
```

### Interactive add

`git add` and `git reset` can be used to stage/unstage a whole file,
but you can use interactive mode to stage by hunk, choosing
yes or no for each hunk.

``` bash
git add -p myfile.py
```

``` python
+import matplotlib
+import numpy as np
#Stage this hunk [y,n,a,d,/,j,J,g,e,?]?
```

## SSH keys and GitHub

Classroom exercise: If you haven't already, you should set things up so that you don't have to keep typing in your
password whenever you interact with GitHub via the command line.

You can do this with an "ssh keypair". You may have created a keypair in the
Software Carpentry shell training. Go to the [ssh settings
page](https://github.com/settings/ssh) on GitHub and upload your public key by
copying the content from your computer. (Probably at .ssh/id_rsa.pub)

If you have difficulties, the instructions for this are [on the GitHub
website](https://help.github.com/articles/generating-ssh-keys). 

## Rebasing

### Rebase vs merge

A git *merge* is only one of two ways to get someone else's work into yours.
The other is called a rebase.

In a merge, a revision is added, which brings the branches together. Both histories are retained.
In a rebase, git tries to work out

> What would you need to have done, to make your changes, if your colleague had already made theirs?

Git will invent some new revisions, and the result will be a repository with an apparently linear history.

### An example rebase

We've built a repository to help visualise the difference between a merge and a rebase, at https://github.com/UCL-RITS/wocky_rebase/blob/master/wocky.md .

The initial state of both collaborators is a text file, wocky.md:

```
It was clear and cold,
and the slimy monsters
```

On the master branch, a second commit ('Dancing') has been added:

```
It was clear and cold,
and the slimy monsters
danced and spun in the waves
```

On the "Carollian" branch, a commit has been added translating the initial state into Lewis Caroll's language:

```
'Twas brillig,
and the slithy toves
```

So the logs look like this:

```bash
git log --oneline --graph master
```

```
* 2a74d89 Dancing
* 6a4834d Initial state
```

```bash
git log --oneline --graph carollian
```

```
* 2232bf3 Translate into Caroll's language
* 6a4834d Initial state
```

If we now **merge** carollian into master, the final state will include both changes:

```
'Twas brillig,
and the slithy toves
danced and spun in the waves
```


But the graph shows a divergence and then a convergence:

```
git log --oneline --graph
```

```
*   b41f869 Merge branch 'carollian' into master_merge_carollian
|\
| * 2232bf3 Translate into Caroll's language
* | 2a74d89 Dancing
|/
* 6a4834d Initial state
```

But if we **rebase**, the final content of the file is still the same, but the graph is different:

``` bash
git log --oneline --graph master_rebase_carollian
```

```
* df618e0 Dancing
* 2232bf3 Translate into Caroll's language
* 6a4834d Initial state
```

To trigger the rebase, we did:
    
``` bash
git checkout master
git rebase carollian
```

If this had been a remote, we would merge it with:
    
``` bash
git pull --rebase
```

### Fast Forwards

If we want to continue with the translation, and now want to merge the rebased branch into the carollian branch, 
we get:

``` bash
Updating 2232bf3..df618e0
Fast-forward
 wocky.md | 1 +
 1 file changed, 1 insertion(+)
```

The rebased branch was **rebased on** the carollian branch, so this merge was just a question of updating *metadata*
to redefine the branch label: a "fast forward".

### Rebasing pros and cons

Some people like the clean, apparently linear history that rebase provides.

But *rebase rewrites history*.

If you've already pushed, or anyone else has got your changes, things will get screwed up.

If you know your changes are still secret, it might be better to rebase to keep the history clean.
If in doubt, just merge.

## Squashing

A second use of the `git rebase` command, is to rebase your work on top of one of *your own* earlier commits,
in interactive mode, to "squash" several commits that should really be one:

``` bash
git log
```

```
ea15 Some good work
ll54 Fix another typo
de73 Fix a typo
ab11 A great piece of work
cd27 Initial commit
```

### Using rebase to squash

If we type 

``` bash
git rebase -i ab11 #OR HEAD^^
```

an edit window pops up with:

```
pick cd27 Initial commit
pick ab11 A great piece of work
pick de73 Fix a typo
pick ll54 Fix another typo
pick ea15 Some good work

# Rebase 60709da..30e0ccb onto 60709da
#
# Commands:
#  p, pick = use commit
#  e, edit = use commit, but stop for amending
#  s, squash = use commit, but meld into previous commit
```

We can rewrite select commits to be merged, so that the history is neater before we push.
This is a great idea if you have lots of trivial typo commits.

```
pick cd27 Initial commit
pick ab11 A great piece of work
squash de73 Fix a typo
squash ll54 Fix another typo
pick ea15 Some good work
```

save the interactive rebase config file, and rebase will build a new history:

``` bash
git log
```

```
de82 Some good work
fc52 A great piece of work
cd27 Initial commit
```

Note the commit hash codes for 'Some good work' and 'A great piece of work' have changed, 
as the change they represent has changed.

## Debugging

### Debugging With Git Bisect

You can use

``` bash
git bisect
```

to find out which commit caused a bug.

Automated Bisect

In [ ]:
git bisect run py.test

### An example repository

In a nice open source example, I found an arbitrary exemplar on github

In [231]:
os.chdir(git_dir)

In [232]:
%%bash
rm -rf bisectdemo
git clone git@github.com:shawnsi/bisectdemo.git

Cloning into 'bisectdemo'...


In [233]:
bisect_dir=os.path.join(git_dir,'bisectdemo')
os.chdir(bisect_dir)

In [234]:
%%bash
python squares.py 2 # 4

4


This has been set up to break itself at a random commit, and leave you to use
bisect to work out where it has broken:

In [235]:
%%bash 
./breakme.sh > break_output

error: branch 'buggy' not found.
Switched to a new branch 'buggy'


Which will make a bunch of commits, of which one is broken, and leave you in the broken final state

In [236]:
python squares.py 2 # Error message

SyntaxError: invalid syntax (<ipython-input-236-69f578907137>, line 1)

In [241]:
### Bisecting manually

In [237]:
%%bash
git bisect start
git bisect bad # We know the current state is broken
git checkout master
git bisect good # We know the master branch state is OK

Your branch is up-to-date with 'origin/master'.
Bisecting: 500 revisions left to test after this (roughly 9 steps)
[97b8922da658f4adbe0e8d1262062d171e2f1683] Comment 500


Switched to branch 'master'


Bisect needs one known good and one known bad commit to get started

### Solving Manually

``` bash
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
python squares.py 2 # Crash
git bisect bad
python squares.py 2 # Crash
git bisect bad
python squares.py 2 # Crash
git bisect bad
python squares.py 2 #Crash
git bisect bad
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
python squares.py 2 # 4
git bisect good
```


And eventually:

``` bash
git bisect good
    Bisecting: 0 revisions left to test after this (roughly 0 steps)

python squares.py 2
    4

git bisect good
2777975a2334c2396ccb9faf98ab149824ec465b is the first bad commit
commit 2777975a2334c2396ccb9faf98ab149824ec465b
Author: Shawn Siefkas <shawn.siefkas@meredith.com>
Date:   Thu Nov 14 09:23:55 2013 -0600

    Breaking argument type

```

``` bash
git bisect end
```

### Solving automatically

If we have an appropriate unit test, we can do all this automatically:

In [242]:
%%bash
git bisect start
git bisect bad HEAD # We know the current state is broken
git bisect good master # We know master is good
git bisect run python squares.py 2

Bisecting: 500 revisions left to test after this (roughly 9 steps)
[97b8922da658f4adbe0e8d1262062d171e2f1683] Comment 500
running python squares.py 2
4
Bisecting: 250 revisions left to test after this (roughly 8 steps)
[f10ce756d4cce15135566ecfc2c5d85004ab191f] Comment 750
running python squares.py 2
4
Bisecting: 125 revisions left to test after this (roughly 7 steps)
[ec9e48e76383217dc2dd5dd8bb11ac3825e473f5] Comment 875
running python squares.py 2
4
Bisecting: 62 revisions left to test after this (roughly 6 steps)
[3c1afca389c1cbb2cf7bfea3afc04accf17265e5] Comment 937
running python squares.py 2
Bisecting: 31 revisions left to test after this (roughly 5 steps)
[13dae55a5d37c6bbe477f555e7c829607e1dbbe9] Comment 905
running python squares.py 2
Bisecting: 15 revisions left to test after this (roughly 4 steps)
[76dd89679d416901181e1c70dcf8b674f3c72f2e] Comment 889
running python squares.py 2
Bisecting: 7 revisions left to test after this (roughly 3 steps)
[2777975a2334c2396ccb9faf98ab149

Traceback (most recent call last):
  File "squares.py", line 9, in <module>
    print(integer**2)
TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'
Traceback (most recent call last):
  File "squares.py", line 9, in <module>
    print(integer**2)
TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'
Traceback (most recent call last):
  File "squares.py", line 9, in <module>
    print(integer**2)
TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'
Traceback (most recent call last):
  File "squares.py", line 9, in <module>
    print(integer**2)
TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'


Boom!

## GitHub pages

### Yaml Frontmatter

GitHub will publish repositories containing markdown as web pages, automatically. 

You'll need to add this content:

> ```
>    ---
>    ---
> ```

A pair of lines with three dashes, to the top of each markdown file. This is how GitHub knows which markdown files to make into web pages.
[Here's why](https://github.com/mojombo/jekyll/wiki/YAML-Front-Matter) for the curious. 

In [247]:
%%writefile index.md
---
title: Github Pages Example
---
Mountains and Lakes in the UK
===================

Engerland is not very mountainous.
But has some tall hills, and maybe a mountain or two depending on your definition.

Overwriting index.md


In [248]:
%%bash
git commit -am "Add github pages YAML frontmatter"

[gh-pages a37f445] Add github pages YAML frontmatter
 1 file changed, 6 insertions(+), 2 deletions(-)


### The gh-pages branch

GitHub creates github pages when you use a special named branch.

This is best used to create documentation for a program you write, but you can use it for anything.

In [249]:
os.chdir(working_dir)

In [250]:
%%bash

git checkout -b gh-pages
git push -u origin gh-pages

Branch gh-pages set up to track remote branch gh-pages from origin.


fatal: A branch named 'gh-pages' already exists.
To git@github.com:UCL/github-example.git
   5e25fe8..a37f445  gh-pages -> gh-pages


The first time you do this, GitHub takes a few minutes to generate your pages. 

The website will appear at `http://username.github.io/repositoryname`, for example:

http://UCL.github.io/github-example/

### UCL layout for GitHub pages

You can use GitHub pages to make HTML layouts, here's an [example of how to do it](http://github.com/UCL/ucl-github-pages-example), 
and [how it looks](http://ucl.github.com/ucl-github-pages-example). We won't go into the detail of this now, 
but after the class, you might want to try this.